In [3]:
import conv
#import data_helpers
import tensorflow as tf
import numpy as np
import os
import time
import datetime
#import data_helpers
import nltk
import word2vec
import re
import itertools
from collections import Counter
from gensim.models import word2vec
from tensorflow.contrib import learn


## Downloading and splitting

In [2]:
import data_helpers

tweets='TREC_all.txt'
labels='label_all.txt'

X,y=data_helpers.load_tweets(tweets),data_helpers.load_1dsentiment(labels)

## Words representing

In [4]:
# Build vocabulary
max_document_length = max([len(x.split(" ")) for x in X])
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x_bag = np.array(list(vocab_processor.fit_transform(X)))
y_text = np.array([[sent, 1-sent] for sent in y])

In [ ]:
#word2vec


## Splitting

In [18]:
from sklearn.model_selection import train_test_split

x_train, x_dev, y_train, y_dev = train_test_split(x_bag, y_text, test_size=0.33, random_state=42)

print ("Train class frequencies:\t", [col.nonzero()[0].shape[0] for col in y_train.transpose()])
print ("Validation class frequencies:\t", [col.nonzero()[0].shape[0] for col in y_dev.transpose()])
print ("Constant classifier's validation accuracy:\t", [col.nonzero()[0].shape[0] for col in y_dev.transpose()][1] * 1. / y_test.shape[0])

Train class frequencies:	 [1639, 4653]
Validation class frequencies:	 [782, 2318]
Constant classifier's validation accuracy:	 0.747741935483871


## Train CNN

In [10]:
# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 50, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 20, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")


In [11]:
FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))


Parameters:
BATCH_SIZE=50
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.1
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=128
EVALUATE_EVERY=100
FILTER_SIZES=3,4,5
L2_REG_LAMBDA=0.0
NUM_CHECKPOINTS=5
NUM_EPOCHS=20
NUM_FILTERS=128


In [19]:
with tf.Graph().as_default():
    sess = tf.Session()
    with sess.as_default():
        
        cnn = conv.TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=2,
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters)
        
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-4)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        
        
        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))
 
        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)
 
        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph_def)
 
        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph_def)
        
        
        # Checkpointing
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        # Tensorflow assumes this directory already exists so we need to create it
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.all_variables())
        
        
        sess.run(tf.initialize_all_variables())
        
        
        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
                cnn.input_x: x_batch,
                cnn.input_y: y_batch,
                cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)
            
            
        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)
                
        # Generate batches
        batches = data_helpers.batch_iter(
            zip(x_train, y_train), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))




Writing to /Users/helenahuddy/CNN/runs/1508156672

Instructions for updating:
Please use tf.global_variables instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
2017-10-16T15:24:33.786149: step 1, loss 1.99794, acc 0.48
2017-10-16T15:24:33.965246: step 2, loss 1.67133, acc 0.62
2017-10-16T15:24:34.144599: step 3, loss 1.29331, acc 0.56
2017-10-16T15:24:34.253115: step 4, loss 1.08593, acc 0.64
2017-10-16T15:24:34.372892: step 5, loss 1.40711, acc 0.6
2017-10-16T15:24:34.487709: step 6, loss 1.44321, acc 0.56
2017-10-16T15:24:34.605171: step 7, loss 1.87714, acc 0.5
2017-10-16T15:24:34.714060: step 8, loss 1.65081, acc 0.5
2017-10-16T15:24:34.827519: step 9, loss 1.05365, acc 0.64
2017-10-16T15:24:34.937492: step 10, loss 1.2275, acc 0.64
2017-10-16T15:24:35.049588: step 11, loss 1.08502, acc 0.76
2017-10-16T15:24:35.157937: step 12, loss 1.07223, acc 0.74
2017-10-16T15:24:35.267158: step 13, loss 1.33893, acc 0.66
2017-10-16T15:24:35.380297: step 14, loss

KeyboardInterrupt: 

In [20]:
from tensorflow.contrib.rnn import GRUCell
from tensorflow.python.ops.rnn import bidirectional_dynamic_rnn as bi_rnn
from tensorflow.contrib.layers import fully_connected

In [24]:
EMBED_DIM = 300
HIDDEN_SIZE = 300
NUM_CLASSES = 2

tf.reset_default_graph()

batch_ph   = tf.placeholder(tf.int32, [None, None])
target_ph  = tf.placeholder(tf.float32, [None, NUM_CLASSES])
seq_len_ph = tf.placeholder(tf.int32, [None])
keep_prob_ph = tf.placeholder(tf.float32)

embeddings_ph = tf.placeholder(tf.float32, [len(vocab_processor.vocabulary_), EMBED_DIM])
embeddings_var = tf.Variable(tf.constant(0., shape=[len(vocab_processor.vocabulary_), EMBED_DIM]), trainable=False)
init_embeddings = embeddings_var.assign(embeddings_ph)
batch_embedded = tf.nn.embedding_lookup(embeddings_var, batch_ph)
    
# Bi-RNN layers
outputs,_ = bi_rnn(GRUCell(HIDDEN_SIZE), GRUCell(HIDDEN_SIZE),
                   inputs=batch_embedded,sequence_length=seq_len_ph, dtype=tf.float32, scope="bi_rnn1")  
outputs = tf.concat(outputs, 2)
# outputs,_ = bi_rnn(GRUCell(HIDDEN_SIZE), GRUCell(HIDDEN_SIZE),
#                          inputs=outputs,sequence_length=seq_len_ph, dtype=tf.float32, scope="bi_rnn2")
# outputs = tf.concat(outputs, 2)

# Last output of Bi-RNN
output = outputs[:,0,:]

# Dropout
drop = tf.nn.dropout(output, keep_prob_ph)

# Fully connected layer
W = tf.Variable(tf.truncated_normal([HIDDEN_SIZE * 2, NUM_CLASSES], stddev=0.1), name="W")
b = tf.Variable(tf.constant(0., shape=[NUM_CLASSES]), name="b")
y_hat = tf.nn.xw_plus_b(drop, W, b, name="scores")

In [25]:
# Adam parameters
LEARNING_RATE = 1e-3
EPSILON = 1e-5
BETA1 = 0.9
BETA2 = 0.9
# L2 regularization coefficient
BETA = 0

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_hat, labels=target_ph),
                               name="cross_entropy")
l2_loss = tf.nn.l2_loss(W, name="l2_loss")
loss = cross_entropy + l2_loss * BETA
optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE, beta1=BETA1, beta2=BETA2,
                                   epsilon=EPSILON).minimize(loss)
# optimizer = tf.train.MomentumOptimizer(learning_rate=1e-1, momentum=0.1).minimize(loss)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(target_ph, 1), tf.argmax(y_hat, 1)), tf.float32))

In [30]:
def batch_generator(X, y, batch_size):
    size = X.shape[0]
    X_copy = X.copy()
    y_copy = y.copy()
    indices = np.arange(size)
    np.random.shuffle(indices)
    X_copy = X_copy[indices]
    y_copy = y_copy[indices]
    i = 0
    while True:
        if i + batch_size <= size:
            yield X_copy[i:i + batch_size], y_copy[i:i + batch_size]
            i += batch_size
        else:
            i = 0
            indices = np.arange(size)
            np.random.shuffle(indices)
            X_copy = X_copy[indices]
            y_copy = y_copy[indices]
            continue



In [76]:
#%%time


from sklearn.model_selection import StratifiedKFold, KFold

FOLDS = 10

DROPOUT = 0.5  # Probability of keeping a neuron
BATCH_SIZE = 256
EPOCHS = 10


skf = KFold(FOLDS, shuffle=True, random_state=42)

results = []

for train_index, test_index in skf.split(x_bag, y_text):
    X_tr, X_val = np.array([x_bag[i] for i in train_index]), np.array([x_bag[i] for i in test_index])
    y_tr, y_val = np.array([y_text[i] for i in train_index]), np.array([y_text[i] for i in test_index])
    
    train_batch_generator = batch_generator(X_tr, y_tr, BATCH_SIZE)

    loss_tr_l = []
    loss_val_l = []
    ce_tr_l = []  # Cross-entropy
    ce_val_l = []
    acc_tr_l = []  # Accuracy
    acc_val_l = []
    f_macro_tr_l = []
    f_macro_val_l = []
    f_fair_tr_l = []
    f_fair_val_l = []

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        #sess.run(init_embeddings, feed_dict={embeddings_ph: embeddings})
        print ("Start learning...")
        for epoch in range(EPOCHS):
            for i in range(int(X_tr.shape[0] / BATCH_SIZE)):
                x_batch, y_batch = next(train_batch_generator)
                seq_len_tr = np.array([list(x).index(eos_id) + 1 for x in x_batch])
                sess.run(optimizer, feed_dict={batch_ph: x_batch, target_ph: y_batch,
                                               seq_len_ph: seq_len_tr, keep_prob_ph: DROPOUT})

            y_pred_tr, ce_tr, loss_tr, acc_tr = sess.run([y_hat, cross_entropy, loss, accuracy],
                                                  feed_dict={batch_ph: x_batch, target_ph: y_batch, 
                                                             seq_len_ph: seq_len_tr, keep_prob_ph: 1.0})
            
            y_pred_val, ce_val, loss_val, acc_val = [], 0, 0, 0
            num_val_batches = X_val.shape[0] / BATCH_SIZE
            for i in range(num_val_batches):
                x_batch_val, y_batch_val = X_val[i * BATCH_SIZE : (i + 1) * BATCH_SIZE],\
                                           y_val[i * BATCH_SIZE : (i + 1) * BATCH_SIZE]
                seq_len_val = np.array([list(x).index(eos_id) + 1 for x in x_batch_val])
                y_pred_val_, ce_val_, loss_val_, acc_val_ = sess.run([y_hat, cross_entropy, loss, accuracy],
                                                             feed_dict={batch_ph: x_batch_val, target_ph: y_batch_val,
                                                                        seq_len_ph: seq_len_val, keep_prob_ph: 1.0})
                y_pred_val += list(y_pred_val_)
                ce_val += ce_val_
                loss_val += loss_val_
                acc_val += acc_val_
            
            y_pred_val = np.array(y_pred_val)
            ce_val /= num_val_batches
            loss_val /= num_val_batches
            acc_val /= num_val_batches

            y_pred_tr = np.array([cls2probs(cls) for cls in np.argmax(y_pred_tr, 1) - 1])
            y_pred_val = np.array([cls2probs(cls) for cls in np.argmax(y_pred_val, 1) - 1])
            f_macro_tr, f_micro_tr = f_macro(y_batch, y_pred_tr), f_micro(y_batch, y_pred_tr)
            f_macro_val, f_micro_val = f_macro(y_val[:num_val_batches * BATCH_SIZE], y_pred_val),\
                                       f_micro(y_val[:num_val_batches * BATCH_SIZE], y_pred_val)
        

            loss_tr_l.append(loss_tr)
            loss_val_l.append(loss_val)
            ce_tr_l.append(ce_tr)
            ce_val_l.append(ce_val)
            acc_tr_l.append(acc_tr)
            acc_val_l.append(acc_val)
            f_macro_tr_l.append(f_macro_tr)
            f_macro_val_l.append(f_macro_val)

            clear_output(wait=True)
            print ("epoch: {}".format(epoch))
            print ("\t Train loss: {:.3f}\t ce: {:.3f}\t acc: {:.3f}\t f_macro: {:.3f}".format(
                loss_tr, ce_tr, acc_tr, f_macro_tr))
            print ("\t Valid loss: {:.3f}\t ce: {:.3f}\t acc: {:.3f}\t f_macro: {:.3f}".format(
                loss_val, ce_val, acc_val, f_macro_val))

            """plt.figure(figsize=(15,10))
            plt.plot(ce_tr_l, color='blue', label='ce_tr')
            plt.plot(ce_val_l, color='red', label='ce_val')        
            plt.plot(f_macro_val_l, color='green', label='f_macro_val')
            plt.xlim(0, EPOCHS - 1)
            plt.ylim(0, 1)
            plt.legend()
            plt.show()"""
            
    results.append([max(acc_val_l), max(f_macro_val_l)])

Start learning...


NameError: name 'eos_id' is not defined

In [83]:
from tensorflow.contrib import rnn

In [105]:
learning_rate = 0.001
training_steps = 10000
batch_size = 50
display_step = 200

# Network Parameters
num_input = 39 # MNIST data input (img shape: 28*28)
timesteps = 39 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 2 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [106]:
weights = {
    # Hidden layer weights => 2*n_hidden because of forward + backward cells
    'out': tf.Variable(tf.random_normal([2*num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [107]:
def BiRNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, num_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, num_input)
    x = tf.unstack(x, timesteps, 1)

    # Define lstm cells with tensorflow
    # Forward direction cell
    lstm_fw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    # Backward direction cell
    lstm_bw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    try:
        outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                              dtype=tf.float32)
    except Exception: # Old TensorFlow version only returns outputs not states
        outputs = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                        dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [108]:
logits = BiRNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

ValueError: Attempt to have a second RNNCell use the weights of a variable scope that already has weights: 'bidirectional_rnn/fw/basic_lstm_cell'; and the cell was not constructed as BasicLSTMCell(..., reuse=True).  To share the weights of an RNNCell, simply reuse it in your second calculation, or create a new one with the argument reuse=True.

In [100]:
# Start training
with tf.Session() as sess:
    train_batch_generator = batch_generator(X_tr, y_tr, BATCH_SIZE)
    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps+1):
        
        batch_x, batch_y = next(train_batch_generator)
        # Reshape data to get 28 seq of 28 elements
        #batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = len(x_dev)
    test_data = x_dev.reshape((-1, timesteps, num_input))
    test_label = y_dev
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_7' with dtype float
	 [[Node: Placeholder_7 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Placeholder_7', defined at:
  File "/Applications/anaconda/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Applications/anaconda/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Applications/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Applications/anaconda/lib/python3.5/site-packages/traitlets/config/application.py", line 653, in launch_instance
    app.start()
  File "/Applications/anaconda/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/Applications/anaconda/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 162, in start
    

In [91]:
len(x_train)

6292

In [28]:
import Stemmer
stemmer = Stemmer.Stemmer('russian')
#stemmer.stemWord('ковриков')

lists=[]
for i in all_examples:
    lists.append([stemmer.stemWord(word) for word in i.split()])

## Word2vec

In [38]:
#Train the model Word2Vec
num_features = 300                  
min_word_count = 40                          
num_workers = 4       
context = 10                                                                                             
downsampling = 1e-3   


model = word2vec.Word2Vec(lists, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

model.init_sims(replace=True)
model_name = "300features_40minwords_10context"
model.save(model_name)

In [40]:
model.most_similar('банк')

[('ипотек', 0.9579096436500549),
 ('услов', 0.9547955989837646),
 ('кредитован', 0.9546328186988831),
 ('лиц', 0.952491044998169),
 ('автокред', 0.9520655274391174),
 ('отзыв', 0.9492961168289185),
 ('физическ', 0.9450693130493164),
 ('ипотечн', 0.9447919130325317),
 ('рассчита', 0.9425774812698364),
 ('оформлен', 0.9424964189529419)]

In [50]:
def makeVec(words, model, num_features):   
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [44]:
#Обучить на куче файлов
""""class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()
 
sentences = MySentences('123') # a memory-friendly iterator
model2 = word2vec.Word2Vec(sentences)""""

In [3]:
""""from gensim.models.keyedvectors import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('web_russe.model.bin', binary=True) 
word_vectors.vocab""""

In [106]:
x_text=[makeVec(i,model, num_features) for i in all_examples]
x_text=np.array(x_text)